In [4]:
!python -m venv ST2

/bin/bash: line 1: ST2ScriptsActivate: command not found


In [11]:
#!ST2/bin/activate
!source ST2/bin/activate
#!ST2/Scripts/Activate

In [12]:
!pip install spacy

  Using cached spacy-3.7.4-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (27 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.10-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.8-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (8.4 kB)
  Using cached preshed-3.0.9-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.3-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (15 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.4.8-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.3.4-py3-none-any.whl.metadata (4.7 kB)
  Using cached typer-0.9.

In [24]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 434.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 845.9 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.9/783.9 kB 4.9 MB/s eta 0:00:0000:0100:01


In [25]:
import csv
import sys

from spacy.pipeline._edit_tree_internals.edit_trees import EditTrees
from spacy.strings import StringStore
import sacrebleu

In [15]:
edit_trees = EditTrees(StringStore([]))

change_to_trees = {}

Setting the path to the mayan files

In [20]:
train_path = "../data/maya-train.tsv"
dev_path = "../data/maya-dev.tsv"
dev_prediction_path = "maya-dev-prediction.tsv"

In [21]:
with open(train_path, newline="", encoding="utf-8") as tf:
    tcsv = csv.DictReader(tf, delimiter="\t")

    for row in tcsv:
        tree = edit_trees.add(
            row["Source"], row["Target"]
        )  # returns old tree id when resulting tree already exists

        change = row["Change"]

        if change not in change_to_trees:
            change_to_trees[change] = {}

        if tree in change_to_trees[change]:
            change_to_trees[change][tree] += 1
        else:
            change_to_trees[change][tree] = 1

In [22]:
with open(dev_path, newline="", encoding="utf-8") as ifile, open(
    dev_prediction_path, "w", newline="", encoding="utf-8"
) as ofile:
    icsv = csv.DictReader(ifile, delimiter="\t")
    fieldnames = icsv.fieldnames
    fieldnames.append("Predicted Target")

    ocsv = csv.DictWriter(ofile, fieldnames=fieldnames, delimiter="\t")
    ocsv.writeheader()

    for row in icsv:
        prediction = None
        change = row["Change"]

        if change in change_to_trees:
            trees = change_to_trees[change]
            for tree in sorted(trees, key=lambda t: trees[t], reverse=True):
                prediction = edit_trees.apply(tree, row["Source"])
                if prediction:
                    break

        if prediction:
            row["Predicted Target"] = prediction
        else:
            row["Predicted Target"] = row["Source"]

        ocsv.writerow(row)

In [26]:
references = []
predictions = []

with open(dev_prediction_path, newline="", encoding="utf-8") as ifile:
    icsv = csv.DictReader(ifile, delimiter="\t")
    for row in icsv:
        references.append(row["Target"])
        predictions.append(row["Predicted Target"])

accuracy = (
    sum([int(r == p) for r, p in zip(references, predictions)]) / len(references) * 100
)
bleu = sacrebleu.corpus_bleu(predictions, [references]).format(score_only=True)
chrf = sacrebleu.corpus_chrf(predictions, [references]).format(score_only=True)

In [27]:
print(f"{dev_prediction_path}: Accuracy={accuracy:.02f}, BLEU={bleu}, ChrF={chrf}")

maya-dev-prediction.tsv: Accuracy=26.17, BLEU=52.38, ChrF=78.72
